In [1]:
import pickle, pandas as pd


In [2]:
data = pd.read_csv("validation_for students.csv", index_col='index')
data.index.min()
data['index1'] = data.index
data.sample()

,store_ID,day_of_week,date,nb_customers_on_day,open,promotion,state_holiday,school_holiday,index1
index,,,,,,,,,
635498,682,7,25/05/2014,1304,1,0,0,0,635498


# Feature Enginnering

In [3]:
df = data[(data['open']!=0)]
df = df[(df['nb_customers_on_day']!=0)]
df['month'] = pd.to_datetime(df['date']).dt.month

df['state_holiday'] = df['state_holiday'].replace({'a':1, 'b':1, 'c':1})
# df['year'] = pd.to_datetime(df['date']).dt.year
# cust_bin = [0,500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000]
cust_bin =  [0, 196, 392, 588, 784, 980, 1176, 1372, 1568, 1764, 1960, 2156, 2352, 2548, 2744, 2940, 3136, 3332, 3528, 3724, 3920, 4116, 4312, 4508, 4704, 4900, 5096, 5292, 5488, 5684, 5800] #[0, 189, 378, 567, 756, 945, 1134, 1323, 1512, 1701, 1890, 2079, 2268, 2457, 2646, 2835, 3024, 3213, 3402, 3591, 3780, 3969, 4158, 4347, 4536, 4725, 4914, 5103, 5292, 5481, 5670]
cust_label = [0,1,2,3,4,5,6,7,8,9,10,11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,25,26,27,28,29]
df['cust_bin'] = pd.cut(df['nb_customers_on_day'], bins=cust_bin, labels=cust_label)

store_summary = df.groupby('store_ID')[['nb_customers_on_day']].mean().reset_index()
# print(store_summary.sample(5))
store_label = [0,1,2,3,4,5,6,7,8,9,10,11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,25,26,27,28,29]
store_bins=  [0, 125, 250, 375, 500, 625, 750, 875, 1000, 1125, 1250, 1375, 1500, 1625, 1750, 1875, 2000, 2125, 2250, 2375, 2500, 2625, 2750, 2875, 3000, 3125, 3250, 3375, 3500, 3625, 3800] #[0, 120, 240, 360, 480, 600, 720, 840, 960, 1080, 1200, 1320, 1440, 1560, 1680, 1800, 1920, 2040, 2160, 2280, 2400, 2520, 2640, 2760, 2880, 3000, 3120, 3240, 3360, 3480, 3600]
store_summary.columns = ['store_ID', 'avg_cust'] 
store_summary['store_bin'] = pd.cut(store_summary['avg_cust'], bins=store_bins, labels=store_label)

df_merge = pd.merge(df, store_summary, on= 'store_ID', how='left')
df_merge = df_merge.drop(columns=['date','open','nb_customers_on_day', 'store_ID','avg_cust','school_holiday'])

df_merge['month'] = df_merge['month'].astype('category')
df_merge['cust_bin'] = df_merge['cust_bin'].astype('category')
df_merge['store_bin'] = df_merge['store_bin'].astype('category')
df_merge['store_bin'] = df_merge['store_bin'].cat.codes
df_merge['cust_bin'] = df_merge['cust_bin'].cat.codes
df_merge['month'] = df_merge['month'].cat.codes
df_merge['store_bin'] = df_merge['store_bin'].astype('category')
df_merge['cust_bin'] = df_merge['cust_bin'].astype('category')
df_merge['month'] = df_merge['month'].astype('category')
df_merge = df_merge[['day_of_week', 'promotion', 'state_holiday', 'cust_bin', 'month','store_bin','index1']]

df_merge.sample(5)  #640840

/var/folders/lw/14dpn8b91z99svtdtfqnx09800tlp4/T/ipykernel_46238/3125527608.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['month'] = pd.to_datetime(df['date']).dt.month


,day_of_week,promotion,state_holiday,cust_bin,month,store_bin,index1
39107,6,0,0,3,0,4,246183
550,5,1,0,3,5,5,42681
34010,3,1,0,3,10,5,83753
9928,3,1,0,2,5,3,356100
57543,3,0,0,2,9,4,175668


## Load model

In [4]:
model = pickle.load(open('deepseek_1.pkl','rb'))

In [5]:
y_pred = model.predict(df_merge.drop(columns='index1'))
y_pred

array([6671.49655608, 6101.33139842, 6671.49655608, ..., 5771.03458523,
       7118.42456496, 8017.33309261])

In [6]:
df_merge['sales']=y_pred
df_merge.sample(5)



,day_of_week,promotion,state_holiday,cust_bin,month,store_bin,index1,sales
57213,2,0,0,5,3,9,515978,7469.111640
12189,3,0,0,2,1,4,552407,4497.959228
58186,5,0,0,3,2,6,476119,6163.464111
31648,3,1,0,2,7,3,133437,5142.691646
30593,2,1,0,8,10,9,56447,14106.636575


In [7]:
val_data = df_merge.drop(columns = ['day_of_week', 'promotion', 'state_holiday', 'cust_bin', 'month','store_bin'])


final_val = pd.merge(data, val_data, on='index1', how='left')
final_val.loc[((final_val['sales'].isna()) | (final_val['open']==0) | (df['nb_customers_on_day']==0)),'sales'] = 0

final_val = final_val.drop(columns = ['store_ID', 'day_of_week', 'date', 'nb_customers_on_day', 'open','promotion', 'state_holiday', 'school_holiday'])
final_val.to_csv('validation_data_deepkseek.csv')